In [ ]:
#from helpers.ImageEmbeddings import ImageEmbedding
from img2vec_pytorch import Img2Vec
from PIL import Image
import random
import matplotlib.pyplot as plt
import os #usada apenas para saber a quantidaade de imagens sendo usadas
import re
import torch

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
import pandas as pd
from elasticsearch.helpers import bulk

In [ ]:
index = 'plant_index'
df = pd.read_csv('data/imgs/labels.csv')

In [ ]:
#elastic_client = Elasticsearch(hosts=['http://localhost:9200'], basic_auth=('elastic', '9f-T-qxVj-AeFKIAb45x'))
elastic_client = Elasticsearch(hosts=['http://localhost:9200'])

In [ ]:
#we have to scrape all the folders inside the train folder to get all the images
train_dir = 'data/imgs/train'

file_list = []
for subdir in os.listdir(train_dir):
    subdir_path = os.path.join(train_dir, subdir)
    # make sure the subdirectory is actually a directory
    if os.path.isdir(subdir_path):
        # loop through the files in the subdirectory
        for filename in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, filename)
            # make sure the file is actually a file (not a directory)
            if os.path.isfile(file_path):
                # add the filename (with extension) to the list
                #file_name
                file_list.append(file_path)  

In [ ]:
file_count = len(file_list)

In [ ]:
torch.cuda.is_available()


In [ ]:
print(torch.__version__)

In [ ]:
img2vec = Img2Vec(cuda=True)

In [ ]:
def build_plants(name:str):
  #img_Embedding.generate_embedding(name)
  #vec = img_Embedding.embeddings[name]

  img = Image.open(name)
  vec = img2vec.get_vec(img)

  #getting the data from the field named "labels" from the dataframe where the field "image" is the same as name
  #labels = df.loc[df['image'] == name, 'labels'].values[0]

  name_hash = name.split(".")[0]
  name_hash = re.sub(r'.*\\', '', name_hash)
  print(name_hash)
  name_for_es = name_hash + ".jpg"
  labels = df.loc[df['image'] == name_for_es, 'labels'].values[0]
  
  document={ 
      "name_hash": name_hash,
      "label": labels,
      "embedding": vec,
  }   
  return document

In [ ]:
#looping in the amount of images
actions = []
count = 0
for i in range(file_count):
    file_name_now = str(file_list[i])
    count += 1
    print(f'File {count} of {file_count} - {file_name_now}')
    #resp = elastic_client.index(index=index, body=build_plants(file_name_now))
    #print(resp['result'])
    actions.append({"_index":index,"_source":build_plants(file_name_now)})
    if (i % 100) == 0: 
        print(f"Indexing {i} images")
        response = bulk(elastic_client, actions)
        print(response)
        actions.clear()
    
if len(actions) != 0:
    bulk(elastic_client, actions)
    actions.clear()
print(f"Indexing {i} images")   
